In [8]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
cmx=confusion_matrixes=pd.read_feather("data/confusion_matrixes.feather")

In [5]:
res=results=pd.read_feather("data/results.feather")

In [10]:
logs=pd.read_feather("data/training_logs.feather")